In [ ]:
import torch
import torch.nn as nn
import numpy as np
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
import wandb

from src.neural_net import *
from src.models import *
from prb_def import *


# Data generation

In [2]:
ntrain = 50000
ntest = 1000
batch_size = 500

# Create datasets
train_loader, test_loader = create_data_loaders(ntrain, ntest, batch_size)

# Create the model 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SL_model = MLPWithSTE(insize=2*nx+1, outsize=ny,
                bias=True,
                linear_map=torch.nn.Linear,
                nonlin=nn.ReLU,
                hsizes=[128] * 2)

slack_weight = 1e3
constraint_weight = 1e6
supervised_weight = 1e5
cvx_layer = example_QP(nx=nx, ny=ny, penalty="l1", rho1=slack_weight)

Model = SSL_MIQP_incorporated(SL_model, cvx_layer, nx, ny, device=device)

In [4]:
training_params = {}
training_params['TRAINING_EPOCHS'] = int(1)
training_params['CHECKPOINT_AFTER'] = int(20)
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['PATIENCE'] = 5

Model.train_SL(ground_truth_solver, train_loader, test_loader, training_params)


[epoch 1 | step 1] training loss = 0.1905, validation loss = 0.1155
[epoch 1 | step 20] training loss = 0.0180, validation loss = 0.0188
[epoch 1 | step 40] training loss = 0.0145, validation loss = 0.0140
[epoch 1 | step 60] training loss = 0.0095, validation loss = 0.0095
[epoch 1 | step 80] training loss = 0.0055, validation loss = 0.0090
[epoch 1 | step 100] training loss = 0.0115, validation loss = 0.0090


In [5]:
# Then refine with self-supervised learning
training_params = {}
training_params['TRAINING_EPOCHS'] = int(5)
training_params['CHECKPOINT_AFTER'] = int(20)
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['PATIENCE'] = 10    

slack_weight = 1e3
constraint_weight = 1e6
supervised_weight = 1e5
loss_weights = [0.0, slack_weight, constraint_weight, supervised_weight]
def quad_fcn(x, y, theta): 
    p = theta[:, :nx]
    return (x**2).sum(dim=1) + (p*x).sum(dim=1)
def y_sum_con(y, theta):
    return torch.sum(y, dim=-1) - 1.0  # should be <= 0
y_cons = [y_sum_con]

Model.train_SSL(ground_truth_solver, train_loader, test_loader, training_params, loss_weights, 
            loss_scale=1e2, obj_fcn=quad_fcn, y_cons=y_cons)

[epoch 1 | step 1] validation: loss = 1.0445, obj_val = -60.7921, avg_opt_gap = 38.4097 percent, slack_pen = -0.0000, y_sum_penalty = 0.0000, supervised_loss = 0.1150, 
[epoch 1 | step 20] validation: loss = 1.2897, obj_val = -97.3615, avg_opt_gap = 1.3722 percent, slack_pen = 0.0000, y_sum_penalty = 0.0130, supervised_loss = 0.0120, 
[epoch 1 | step 40] validation: loss = 0.4541, obj_val = -98.7045, avg_opt_gap = 0.0112 percent, slack_pen = 0.0000, y_sum_penalty = 0.0040, supervised_loss = 0.0100, 
[epoch 1 | step 60] validation: loss = 0.1090, obj_val = -96.1942, avg_opt_gap = 2.5560 percent, slack_pen = 0.0000, y_sum_penalty = 0.0000, supervised_loss = 0.0120, 
[epoch 1 | step 80] validation: loss = 0.0658, obj_val = -97.4709, avg_opt_gap = 1.2610 percent, slack_pen = 0.0000, y_sum_penalty = 0.0000, supervised_loss = 0.0073, 
[epoch 1 | step 100] validation: loss = 0.0568, obj_val = -98.1501, avg_opt_gap = 0.5722 percent, slack_pen = 0.0000, y_sum_penalty = 0.0000, supervised_loss =